In [ ]:
!pip install xarray netCDF4 scipy torch numpy matplotlib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.io import loadmat
import xarray as xr
import joblib
from scipy.stats import pearsonr
import seaborn as sns
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')
print('using device:', device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.1 MB/s eta 0:00:00
using device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

data_folder = '/content/drive/MyDrive/DNN_Wildfire/'
mydir = data_folder
print("Files in folder:")
for i in os.listdir(data_folder):
  print(" ", i)

def get_path(filename):
  return os.path.join(data_folder, filename)

nc4_files = sorted([i for i in os.listdir(data_folder) if i.endswith('.nc4')])
mat_files = sorted([i for i in os.listdir(data_folder) if i.endswith('.mat')])

nc4_dataset = {}
for file in nc4_files:
  path = get_path(file)
  ds = xr.open_dataset(path)
  nc4_dataset[file] = ds
  print(f"Loaded: {file}")

mat_dataset = {}
for file in mat_files:
  path = get_path(file)
  ds = loadmat(path)
  mat_dataset[file] = ds
  print(f"Loaded: {file}")

Mounted at /content/drive
Files in folder:
  MC2_burnedFrac.nc4
  JSBACH-SPITFIRE_burnedFrac.nc4
  wildfire_surrogate4.mat
  wildfire_surrogate6.mat
  wildfire_surrogate9.mat
  LPJ-GUESS-GlobFIRM_burnedFrac.nc4
  JULES-INFERNO_burnedFrac.nc4
  wildfire_surrogate7.mat
  wildfire_surrogate8.mat
  wildfire_surrogate2.mat
  wildfire_surrogate5.mat
  wildfire_surrogate14.mat
  wildfire_surrogate11.mat
  wildfire_surrogate1.mat
  wildfire_surrogate10.mat
  CTEM_burnedFrac.nc4
  CLM_burnedFrac.nc4
  LPJ-GUESS-SPITFIRE_burnedFrac.nc4
  ORCHIDEE-SPITFIRE_burnedFrac.nc4
  wildfire_surrogate3.mat
  wildfire_surrogate13.mat
  wildfire_surrogate12.mat
  LPJ-GUESS-SIMFIRE-BLAZE_burnedFrac.nc4
  11_17_own_data
  11_22_transfer_learning
  11_14_results
  wildfire_surrogate1_DNN_softplus.pt
  11_29_transfer_learning
  global_wildfire_base_model.pt
Loaded: CLM_burnedFrac.nc4
Loaded: CTEM_burnedFrac.nc4
Loaded: JSBACH-SPITFIRE_burnedFrac.nc4
Loaded: JULES-INFERNO_burnedFrac.nc4
Loaded: LPJ-GUESS-GlobFIRM

In [ ]:
class DNNWildfire(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.fc1 = nn.Linear(input_dim, 5)
    self.fc2 = nn.Linear(5, 5)
    self.fc3 = nn.Linear(5, 5)
    self.fc4 = nn.Linear(5, 5)
    self.fc5 = nn.Linear(5, 5)
    self.fc6 = nn.Linear(5, 1)
    self.softplus = nn.Softplus()
    for m in self.modules():
      if isinstance(m, nn.Linear):
        nn.init.uniform_(m.weight, -0.05, 0.05)
        nn.init.zeros_(m.bias)

  def forward(self, x, y=None):
    pred = self.softplus(self.fc1(x))
    pred = self.softplus(self.fc2(pred))
    pred = self.softplus(self.fc3(pred))
    pred = self.softplus(self.fc4(pred))
    pred = self.softplus(self.fc5(pred))
    pred = self.fc6(pred)

    if y is not None:
      loss = nn.functional.mse_loss(pred, y)
      return loss, pred
    return pred




In [ ]:
def validate_epoch(net, val_iter, device):
  net.eval()
  total_loss = 0.0
  count = 0
  with torch.no_grad():
    for val_data in val_iter:
      val_data = [ds.to(device) for ds in val_data]
      loss, pred = net(*val_data)
      total_loss += loss.mean().item()
      count += 1
  net.train()
  return total_loss / count

In [ ]:
def train_dnn(net, train_iter, val_iter, lr, epochs, device, decay_steps=1000, decay_rate=0.99):
  net = net.to(device)
  optimizer = torch.optim.Adam(net.parameters(), lr=lr)
  train_loss_per_epoch = []
  val_loss_per_epoch = []
  print_interval = len(train_iter)
  total_iter = epochs * len(train_iter)

  for e in range(epochs):
    net.train()
    total_train_loss_sum = 0.0
    batch_count = 0
    for i, train_data in enumerate(train_iter):
      train_data = [ds.to(device) for ds in train_data]

      loss, pred = net(*train_data)

      total_train_loss_sum += loss.mean().item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      step = i + e * len(train_iter)
      new_lr = lr * (decay_rate ** (step/decay_steps))
      for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr

      if step % print_interval == 0:
        print('iter {} / {}\tLoss:\t{:.6f}'.format(step, total_iter, loss.mean().detach()))
        print('pred:\t {}\n'.format(pred[0].detach().cpu()))
        print('tgt:\t {}\n'.format(train_data[1][0].cpu()))

    if batch_count > 0:
        avg_train_loss = total_train_loss_sum / batch_count
    else:
        avg_train_loss = 0.0
        print(f"WARNING: Fold {e+1} had an empty training batch (batch_count=0).")

    train_loss_per_epoch.append(avg_train_loss)

    val_loss = validate_epoch(net, val_iter, device)
    val_loss_per_epoch.append(val_loss)
  return train_loss_per_epoch, val_loss_per_epoch

def predict_model(net, X, device):
  net = net.to(device)
  net.eval()
  with torch.no_grad():
    X = X.to(device)
    prediction = net(X).cpu().numpy()
  return prediction



In [ ]:
def train_region(region_idx, mydir, device, force_retrain=False):
  matfiles = os.path.join(mydir, f'wildfire_surrogate{region_idx+1}.mat')
  print(f"\n{'='*60}")
  print(f"Processing region {region_idx+1}")
  print('='*60)

  tmp = loadmat(matfiles)
  ELMX = tmp.get('ELMX')
  ELMy = tmp.get('ELMy')

  sc_X = MinMaxScaler(feature_range=(0, 1))
  sc_y = MinMaxScaler(feature_range=(0, 1))

  X = sc_X.fit_transform(ELMX)
  y = sc_y.fit_transform(ELMy.reshape(-1, 1))

  scaler_filename_X = os.path.join(mydir, f"scaler_X{region_idx+1}.mat")
  scaler_filename_y = os.path.join(mydir, f"scaler_y{region_idx+1}.mat")
  joblib.dump(sc_X, scaler_filename_X)
  joblib.dump(sc_y, scaler_filename_y)

  y_1 = np.percentile(y, 33)
  y_2 = np.percentile(y, 66)
  y_3 = np.max(y)
  strata_y = np.full([len(y), 1], 0)
  for j in range(len(y)):
    if y[j] <= y_1:
      strata_y[j] = 1
    elif y[j] <= y_2:
      strata_y[j] = 2
    elif y[j] <= y_3:
      strata_y[j] = 3

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=strata_y, random_state=0)

  train_dataset = TensorDataset(
      torch.tensor(X_train, dtype=torch.float32),
      torch.tensor(y_train, dtype=torch.float32)
  )
  train_iter = DataLoader(train_dataset, batch_size=20, shuffle=True)

  model_path = os.path.join(mydir, f'wildfire_surrogate{region_idx+1}_DNN_softplus.pt')

  net = DNNWildfire(input_dim=X_train.shape[1])

  if os.path.exists(model_path) and not force_retrain:
    print(f"Loading model")
    net.load_state_dict(torch.load(model_path, map_location=device))
  else:
    print(f"Training model")
    loss_list = train_dnn(net, train_iter, lr=0.01, epochs=30, device=device, decay_steps=1000, decay_rate=0.99)
    torch.save(net.state_dict(), model_path)

  X_all = sc_X.transform(ELMX)
  X_all_tensor = torch.tensor(X_all, dtype=torch.float32)
  y_pred = predict_model(net, X_all_tensor, device)

  dnn_y = sc_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1, 360)
  data_y = sc_y.inverse_transform(y.reshape(-1, 1)).reshape(-1, 360)

  return dnn_y, data_y



In [ ]:
def tune_region(region_idx, mydir, device, obs_name='ensemble'):
  print(f"\n{'='*60}")
  print(f"TUNING region {region_idx+1}")
  print(f"{'='*60}")

  matfiles = os.path.join(mydir, f'wildfire_surrogate{region_idx+1}.mat')
  tmp = loadmat(matfiles)
  OBSX = tmp.get('OBSX')
  OBSy_gfed = tmp.get('OBSy')
  OBSy_cci51 = tmp.get('OBSy_cci51')
  OBSy_ccilt11 = tmp.get('OBSy_ccilt11')
  OBSy_mcd64 = tmp.get('OBSy_mcd64')
  OBSy_atlas = tmp.get('OBSy_atlas')

  OBSy = np.mean(np.hstack((OBSy_gfed, OBSy_cci51, OBSy_ccilt11, OBSy_mcd64, OBSy_atlas)), axis=1).reshape(-1, 1)

  scaler_filename_X = os.path.join(mydir, f"scaler_X{region_idx+1}.mat")
  scaler_filename_y = os.path.join(mydir, f"scaler_y{region_idx+1}.mat")
  sc_X = joblib.load(scaler_filename_X)
  sc_y = joblib.load(scaler_filename_y)

  X = sc_X.transform(OBSX)
  y = sc_y.transform(OBSy.reshape(-1, 1))

  y_1 = np.percentile(y, 33)
  y_2 = np.percentile(y, 66)
  y_3 = np.max(y)
  strata_y = np.full([len(y), 1], 0)
  for j in range(len(y)):
    if y[j] <= y_1:
      strata_y[j] = 1
    elif y[j] <= y_2:
      strata_y[j] = 2
    elif y[j] <= y_3:
      strata_y[j] = 3

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=strata_y, random_state=0)
  train_dataset = TensorDataset(
      torch.tensor(X_train, dtype=torch.float32),
      torch.tensor(y_train, dtype=torch.float32)
  )
  train_iter = DataLoader(train_dataset, batch_size=20, shuffle=True)

  base_model_path = os.path.join(mydir, f'wildfire_surrogate{region_idx+1}_DNN_softplus.pt')
  net = DNNWildfire(input_dim=X_train.shape[1])
  print(f"Loading base model: {base_model_path}")
  net.load_state_dict(torch.load(base_model_path, map_location=device))

  tuned_model_path = ''
  lr = 0.001
  decay_steps = 1000
  decay_rate = 0.9

  if region_idx == 4 or region_idx == 8:
    lr = 0.005
    decay_steps = 3000
    decay_rate = 0.99
    tuned_model_path = os.path.join(mydir, f'wildfire_surrogate{region_idx+1}_DNN_softplus_{obs_name}_tuned4.pt')
  elif region_idx == 7:
    lr = 0.005
    decay_steps = 1000
    decay_rate = 0.99
    tuned_model_path = os.path.join(mydir, f'wildfire_surrogate{region_idx+1}_DNN_softplus_{obs_name}_tuned4.pt')
  else:
    tuned_model_path = os.path.join(mydir, f'wildfire_surrogate{region_idx+1}_DNN_softplus_{obs_name}_tuned2.pt')

  print(f"Tuning model with learning rate = {lr}. Saving to: {tuned_model_path}")
  loss_list = train_dnn(net, train_iter, lr=lr, epochs=100, device=device, decay_steps=decay_steps, decay_rate=decay_rate)
  torch.save(net.state_dict(), tuned_model_path)

  #create tune_ensemble.csv files
  X_all_tensor = torch.tensor(X, dtype=torch.float32)
  y_pred = predict_model(net, X_all_tensor, device)

  dnn_y = sc_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1, 120)
  data_y = sc_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1, 120)

  return np.sum(dnn_y, 0), np.sum(data_y, 0)

In [ ]:
def train_model_from_own_dataset(csv_path, mydir, device, lr=0.005, epochs=200, batch_size=8, test_size=0.20, random_state=0, use_log_transform=True, print_every=10):
    print(f"\nLoad data from: {csv_path}")
    tmp = pd.read_csv(csv_path)

    target_name = 'Burned_Area'
    non_feature_cols = ['Date', target_name]
    feature_names = [c for c in tmp.columns if c not in non_feature_cols]

    print(f"Found {len(feature_names)} features: ")
    print(feature_names)

    tmp = tmp.dropna(subset=feature_names + [target_name]).reset_index(drop=True)

    all_dates = pd.to_datetime(tmp['Date'].values)

    X = tmp[feature_names].values.astype(np.float32)
    y = tmp[target_name].values.astype(np.float32).reshape(-1, 1)

    if use_log_transform:
        print("\nApply log1p transform to target")
        y_trans = np.log1p(y)  # log(1 + y)
        transform_method = 'log1p'
    else:
        print("\nNo transform applied to target")
        y_trans = y.copy()
        transform_method = 'none'

    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X_scaled = sc_X.fit_transform(X)
    y_scaled = sc_y.fit_transform(y_trans)

    joblib.dump(sc_X, os.path.join(mydir, "scaler_X_own_dataset.pkl"))
    joblib.dump(sc_y, os.path.join(mydir, "scaler_y_own_dataset.pkl"))
    joblib.dump({'transform_method': transform_method}, os.path.join(mydir, "preprocessing.pkl"))

    y_1 = np.percentile(y, 33)
    y_2 = np.percentile(y, 66)
    y_3 = np.max(y_scaled)
    strata_y = np.full([len(y_scaled), 1], 0)
    for j in range(len(y_scaled)):
      if y_scaled[j] <= y_1:
        strata_y[j] = 1
      elif y_scaled[j] <= y_2:
        strata_y[j] = 2
      elif y_scaled[j] <= y_3:
        strata_y[j] = 3

    X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(
        X_scaled, y_scaled, all_dates, test_size=test_size, stratify=strata_y, random_state=random_state
    )
    print(f"Train/Test sizes: {len(X_train)} / {len(X_test)}")

    batch_size_train = min(batch_size, max(1, len(X_train)))
    batch_size_val = min(batch_size, len(X_test))

    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32)
    )
    val_dataset = TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True, drop_last=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)

    input_dim = X.shape[1]
    net = DNNWildfire(input_dim).to(device)
    print(f"DEBUG: Calculated input_dim: {input_dim}")

    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    # training loop
    train_losses = []
    val_losses = []
    best_val_loss = np.inf
    best_state = None

    for epoch in range(1, epochs + 1):
        net.train()
        running_loss = 0.0
        batch_count = 0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            optimizer.zero_grad()
            loss, pred = net(xb, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss += loss.item()
            batch_count += 1

        epoch_train_loss = running_loss / batch_count if batch_count > 0 else np.nan
        train_losses.append(epoch_train_loss)

        # validation
        net.eval()
        val_running = 0.0
        val_count = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                loss_val, pred_val = net(xb, yb)
                val_running += loss_val.item()
                val_count += 1
        epoch_val_loss = val_running / val_count if val_count > 0 else np.nan
        val_losses.append(epoch_val_loss)

        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_state = net.state_dict()

        if epoch % print_every == 0 or epoch == 1 or epoch == epochs:
            print(f"Epoch {epoch}/{epochs}  TrainLoss={epoch_train_loss:.6f}  ValLoss={epoch_val_loss:.6f}")

    # restore best model
    if best_state is not None:
        net.load_state_dict(best_state)

    # save final model
    model_path = os.path.join(mydir, 'own_dataset_wildfire_model_baseline.pt')
    torch.save(net.state_dict(), model_path)
    print("Saved model to: ", model_path)

    net.eval()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    with torch.no_grad():
        preds_scaled = net(X_test_tensor).cpu().numpy().flatten()

    y_pred_transformed = sc_y.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    y_test_transformed = sc_y.inverse_transform(y_test).flatten()

    if use_log_transform:
        y_pred_actual = np.expm1(y_pred_transformed)
        y_test_actual = np.expm1(y_test_transformed)
    else:
        y_pred_actual = y_pred_transformed
        y_test_actual = y_test_transformed

    r2 = r2_score(y_test_actual, y_pred_actual)
    mse = mean_squared_error(y_test_actual, y_pred_actual)

    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    print(f"\nTest metrics — R2: {r2:.4f}, MSE: {mse:.4f}, MAE: {mae:.4f}")

    test_loss_transformed = mean_squared_error(y_test_transformed, y_pred_transformed)

    # train/val loss plot
    plt.figure(figsize=(8,5))
    plt.plot(np.arange(1, len(train_losses)+1), train_losses, label='Train Loss')
    plt.plot(np.arange(1, len(val_losses)+1), val_losses, label='Val Loss')

    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title(f"Train/Val Loss (R²={r2:.3f})")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plot_path = os.path.join(mydir, 'train_val_loss_baseline.png')
    plt.savefig(plot_path, dpi=200)
    plt.close()
    print("Train/Val loss plot saved to: ", plot_path)

    # scatter plot actual vs pred
    plt.figure(figsize=(6,6))
    plt.scatter(y_test_actual, y_pred_actual, alpha=0.6, s=40, edgecolors='k', linewidths=0.4)
    mn = min(y_test_actual.min(), y_pred_actual.min())
    mx = max(y_test_actual.max(), y_pred_actual.max())
    plt.plot([mn,mx],[mn,mx], 'r--', linewidth=1.5)
    plt.xlabel('Actual Burned Area')
    plt.ylabel('Predicted Burned Area')
    plt.title(f"Test — R²={r2:.3f}")
    plt.grid(alpha=0.3)
    scatter_path = os.path.join(mydir, 'test_scatter_plot_baseline.png')
    plt.savefig(scatter_path, dpi=200)
    plt.close()
    print("Scatter plot saved to: ", scatter_path)

    # line plot actual vs pred
    plot_tmp = pd.DataFrame({
        'Date': dates_test,
        'Actual': y_test_actual,
        'Predicted': y_pred_actual
    })
    plot_tmp = plot_tmp.sort_values(by='Date')
    r2 = r2_score(plot_tmp['Actual'], plot_tmp['Predicted'])
    plt.figure(figsize=(12, 6))
    plt.plot(plot_tmp['Date'], plot_tmp['Actual'], label="Actual Burned Area", color='red', linewidth=2)
    plt.plot(plot_tmp['Date'], plot_tmp['Predicted'], label="Predicted Burned Area", color='blue', linestyle='--', linewidth=1.5)
    plt.xlabel("Date", fontsize=14, fontweight='bold')
    plt.ylabel(f"Burned Area (Mha)", fontsize=14, fontweight='bold')
    plt.title(f"Model Time-Series Comparison on Full Dataset (R²: {r2:.3f})", fontsize=16)
    plt.legend()
    plt.grid(True, alpha=0.3)
    line_path = os.path.join(mydir, 'time_series_performance_baseline.png')
    plt.savefig(line_path, dpi=200)
    plt.close()
    print("Line plot saved to: ", line_path)

    # time-series prediction visualization plot
    N = len(y_test_actual)
    plt.figure(figsize=(12,6))
    plt.plot(range(N), y_test_actual, label='Actual Burned Area', color='blue', linewidth=2)
    plt.plot(range(N), y_pred_actual, label='Predicted Burned Area', color='yellow', linewidth=2)
    plt.title(f"Time-Series Prediction Visualization (R² = {r2:.3f})")
    plt.legend()
    visualization_path = os.path.join(mydir, 'time-series_prediction_visualization_baseline.png')
    plt.savefig(visualization_path, dpi=200)
    plt.close()
    print("Time-series prediction visualization saved to: ", visualization_path)

    history = {'train_losses': train_losses, 'val_losses': val_losses}

    return net, sc_X, sc_y, history, y_test_actual, y_pred_actual

In [ ]:
own_data_folder = '/content/drive/MyDrive/DNN_Wildfire/11_22_transfer_learning'
mydir_own_data = own_data_folder

csv_files_list = sorted([i for i in os.listdir(mydir_own_data) if i.endswith('dataset.csv')])

if len(csv_files_list) > 0:
    own_dataset_csv_filename = csv_files_list[0]
    own_data_csv_path = os.path.join(mydir_own_data, own_dataset_csv_filename)

    print(f"Found dataset: {own_data_csv_path}")

    trained_net, sc_X, sc_y, history, y_test_actual, y_pred_actual = train_model_from_own_dataset(own_data_csv_path, mydir_own_data, device)

else:
    print(f"ERROR: No file ending in 'dataset.csv' was found in {mydir_own_data}")

Found dataset: /content/drive/MyDrive/DNN_Wildfire/11_22_transfer_learning/DeepLearning_Climate_Biomass_Human_Fire_dataset_2001_2024_montly_dataset.csv

Load data from: /content/drive/MyDrive/DNN_Wildfire/11_22_transfer_learning/DeepLearning_Climate_Biomass_Human_Fire_dataset_2001_2024_montly_dataset.csv
Found 16 features: 
['Precipitation', 'Surface_Temperature', 'Wind_Speed', 'VPD', 'Specific_Humidity', 'Soil_Moisture', 'EVI', 'NDVI', 'LAI', 'Cropland_Area', 'Pasture_Area', 'Rangeland_Area', 'Urban_Area', 'Population_Total', 'Population_Rural', 'Population_Urban']

Apply log1p transform to target
Train/Test sizes: 230 / 58
DEBUG: Calculated input_dim: 16
Epoch 1/200  TrainLoss=1.028102  ValLoss=0.933620
Epoch 10/200  TrainLoss=0.414194  ValLoss=0.360134
Epoch 20/200  TrainLoss=0.259237  ValLoss=0.307446
Epoch 30/200  TrainLoss=0.230277  ValLoss=0.313082
Epoch 40/200  TrainLoss=0.231079  ValLoss=0.287010
Epoch 50/200  TrainLoss=0.225837  ValLoss=0.286785
Epoch 60/200  TrainLoss=0.2043

KeyboardInterrupt: 

In [ ]:
#plot for own dataset
csv_files = sorted([i for i in os.listdir(mydir_own_data) if i.endswith('dataset.csv')])
if not csv_files:
  print("ERROR: No CSV file found to plot.")
else:
  csv_path = os.path.join(mydir_own_data, csv_files[0])
  tmp = pd.read_csv(csv_path).dropna()

  target_name = 'Burned_Area'
  non_feature_columns = ['Date', target_name]

  all_columns = tmp.columns.tolist()

  feature_names = [col for col in all_columns if col not in non_feature_columns]

  sc_X = joblib.load(os.path.join(mydir_own_data, "scaler_X_own_dataset.pkl"))
  sc_y = joblib.load(os.path.join(mydir_own_data, "scaler_y_own_dataset.pkl"))

  net = DNNWildfire(input_dim=len(feature_names))
  net.load_state_dict(torch.load(os.path.join(mydir_own_data, "own_dataset_wildfire_model_baseline.pt"), map_location=device))

  X_all = tmp[feature_names].values
  X_all_scaled = sc_X.transform(X_all)
  X_all_tensor = torch.tensor(X_all_scaled, dtype=torch.float32)
  y_pred_scaled = predict_model(net, X_all_tensor, device)

  y_pred_actual = sc_y.inverse_transform(y_pred_scaled)
  y_actual = tmp[target_name].values

  plt.figure(figsize=(8, 8))
  plt.scatter(y_actual, y_pred_actual, alpha=0.3, label="Data Points")
  plt.plot([y_actual.min(), y_actual.max()], [y_actual.min(), y_actual.max()], 'r--', label="Perfect Match (1:1 Line)")
  plt.xlabel(f"Actual {target_name}")
  plt.ylabel(f"Predicted {target_name}")
  plt.title("Model Performance")
  plt.legend()
  plt.savefig(os.path.join(mydir_own_data, "own_data_model_performance_baseline.png"))
  plt.clf()


<Figure size 800x800 with 0 Axes>